# Deep Learning
## Assignment 4

Previously in 2_fullyconnected.ipynb and 3_regularization.ipynb, we trained fully connected networks to classify notMNIST characters.
The goal of this assignment is make the neural network convolutional.

In [53]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [54]:
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [55]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28, 1) (20000, 10)
Validation set (1000, 28, 28, 1) (1000, 10)
Test set (1000, 28, 28, 1) (1000, 10)


In [296]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [306]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph1 = tf.Graph()

with graph1.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [307]:
num_steps = 5001

with tf.Session(graph=graph1) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.488304
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 200: 1.057189
Minibatch accuracy: 64.1%
Validation accuracy: 73.9%
Minibatch loss at step 400: 0.748401
Minibatch accuracy: 76.6%
Validation accuracy: 82.4%
Minibatch loss at step 600: 0.447125
Minibatch accuracy: 92.2%
Validation accuracy: 83.0%
Minibatch loss at step 800: 0.481827
Minibatch accuracy: 81.2%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 0.633618
Minibatch accuracy: 76.6%
Validation accuracy: 84.7%
Minibatch loss at step 1200: 0.438576
Minibatch accuracy: 89.1%
Validation accuracy: 84.9%
Minibatch loss at step 1400: 0.381663
Minibatch accuracy: 90.6%
Validation accuracy: 85.8%
Minibatch loss at step 1600: 0.348965
Minibatch accuracy: 90.6%
Validation accuracy: 85.4%
Minibatch loss at step 1800: 0.322959
Minibatch accuracy: 90.6%
Validation accuracy: 86.8%
Minibatch loss at step 2000: 0.603554
Minibatch accuracy: 82.8%
Validation accuracy: 8

## Problem 1
The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (nn.max_pool()) of stride 2 and kernel size 2.

In [333]:
#Functions of creating variables
def createPlaceholder(shape=None):
    return tf.placeholder(tf.float32, shape)

def createWeights(size=None):
    initial = tf.truncated_normal(size, stddev=0.1)  
    return tf.Variable(initial)  

def createBiases(size=None):
    initial = tf.constant(0.1, shape=size)  
    return tf.Variable(initial)   

def conv2d(x, W):  
  return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='SAME')  
  
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [320]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph2 = tf.Graph()

with graph2.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [64, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, is_drop=False):
    conv = conv2d(data, layer1_weights)
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden_pool = max_pool_2x2(hidden)
    
    #Second convolutional layer
    conv = conv2d(hidden_pool, layer2_weights)
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden_pool = max_pool_2x2(hidden)
    
    #Full connection
    hidden_pool_flat = tf.reshape(hidden_pool, [-1, 64])
    hidden = tf.nn.relu(tf.matmul(hidden_pool_flat, layer3_weights) + layer3_biases)
    
    if is_drop:
        hidden_drop = tf.nn.dropout(hidden, 0.8)  
        hidden = hidden_drop
    #print hidden3.get_shape()
    return tf.matmul(hidden, layer4_weights) + layer4_biases

  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [321]:
num_steps = 5001

with tf.Session(graph=graph2) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.548594
Minibatch accuracy: 10.9%
Validation accuracy: 9.6%
Minibatch loss at step 200: 1.244410
Minibatch accuracy: 54.7%
Validation accuracy: 70.9%
Minibatch loss at step 400: 0.907697
Minibatch accuracy: 79.7%
Validation accuracy: 79.8%
Minibatch loss at step 600: 0.603808
Minibatch accuracy: 85.9%
Validation accuracy: 81.1%
Minibatch loss at step 800: 0.524586
Minibatch accuracy: 79.7%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 0.751902
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Minibatch loss at step 1200: 0.466052
Minibatch accuracy: 85.9%
Validation accuracy: 83.8%
Minibatch loss at step 1400: 0.394429
Minibatch accuracy: 89.1%
Validation accuracy: 83.6%
Minibatch loss at step 1600: 0.218200
Minibatch accuracy: 92.2%
Validation accuracy: 84.5%
Minibatch loss at step 1800: 0.328499
Minibatch accuracy: 92.2%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.748068
Minibatch accuracy: 85.9%
Validation accuracy: 8

In [335]:
batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

#session = tf.InteractiveSession()
graph3 = tf.Graph()

with graph3.as_default():

  # Input data.
  tf_train_dataset = createPlaceholder((64, 28, 28, 1))
  tf_train_labels = createPlaceholder((64, 10))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = createWeights([5, 5, 1, 16])
  layer1_biases = createBiases([16])

  layer2_weights = createWeights([5, 5, 16, 16])
  layer2_biases = createBiases([16])


  layer3_weights = createWeights([64, 64])
  layer3_biases = createBiases([64])
    

  layer4_weights = createWeights([64, 10])
  layer4_biases = createBiases([10])
    

  
  def model(data, is_drop=False):
    conv = conv2d(data, layer1_weights)
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden_pool = max_pool_2x2(hidden)
    
    #Second convolutional layer
    conv = conv2d(hidden_pool, layer2_weights)
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden_pool = max_pool_2x2(hidden)
    
    #Full connection
    hidden_pool_flat = tf.reshape(hidden_pool, [-1, 64])
    hidden = tf.nn.relu(tf.matmul(hidden_pool_flat, layer3_weights) + layer3_biases)
    
    if is_drop:
        hidden_drop = tf.nn.dropout(hidden, 0.8)  
        hidden = hidden_drop
    #print hidden3.get_shape()
    return tf.matmul(hidden, layer4_weights) + layer4_biases

  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [336]:
num_steps = 2001

with tf.Session(graph=graph3) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 200 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.338613
Minibatch accuracy: 1.6%
Validation accuracy: 7.9%
Minibatch loss at step 200: 1.260819
Minibatch accuracy: 56.2%
Validation accuracy: 71.5%
Minibatch loss at step 400: 0.820820
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Minibatch loss at step 600: 0.539862
Minibatch accuracy: 85.9%
Validation accuracy: 82.6%
Minibatch loss at step 800: 0.526470
Minibatch accuracy: 79.7%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.773433
Minibatch accuracy: 76.6%
Validation accuracy: 83.1%
Minibatch loss at step 1200: 0.507488
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Minibatch loss at step 1400: 0.338042
Minibatch accuracy: 90.6%
Validation accuracy: 83.7%
Minibatch loss at step 1600: 0.300924
Minibatch accuracy: 90.6%
Validation accuracy: 83.8%
Minibatch loss at step 1800: 0.310960
Minibatch accuracy: 89.1%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 0.712758
Minibatch accuracy: 81.2%
Validation accuracy: 85

## Problem 2

Try to get the best performance you can using a convolutional net. Look for example at the classic LeNet5 architecture, adding Dropout, and/or adding learning rate decay.

In [338]:
train_size = train_labels.shape[0]
graph4 = tf.Graph()

with graph4.as_default():

    # Input data.
  tf_train_dataset = createPlaceholder((batch_size, image_size, image_size, num_channels))
  tf_train_labels = createPlaceholder((batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = createWeights([patch_size, patch_size, num_channels, depth])
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = createWeights([patch_size, patch_size, depth, depth])
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = createWeights([64, num_hidden])
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = createWeights([num_hidden, num_labels])
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def model(data, is_drop=False):
    #First convolutional layer
    conv = conv2d(data, layer1_weights)
    hidden1 = tf.nn.relu(conv + layer1_biases)
    hidden1_pool = max_pool_2x2(hidden1)
    #Second convolutional layer
    conv = conv2d(hidden1_pool, layer2_weights)
    hidden2 = tf.nn.relu(conv + layer2_biases)
    hidden2_pool = max_pool_2x2(hidden2)
    
    reshape = tf.reshape(hidden2_pool, [-1, 64])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    if is_drop:
        hidden3_drop = tf.nn.dropout(hidden3, 0.8)  
        hidden3 = hidden3_drop
    #print hidden3.get_shape()
    return tf.matmul(hidden3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                  tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                  tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                  tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases)) 
  # Add the regularization term to the loss.
  loss += 5e-5 * regularizers  

  #Decaying learning rate  
  batch = tf.Variable(0, tf.float32)
  learning_rate = tf.train.exponential_decay(
      0.01,                # Base learning rate.
      batch_size * batch,  # Current index into the dataset.
      train_size,          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
  
  # Optimizer.
  optimizer = tf.train.MomentumOptimizer(learning_rate,
                                         0.9).minimize(loss,
                                                       global_step=batch)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [339]:
num_steps = 5001

with tf.Session(graph=graph4) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.799389
Minibatch accuracy: 9.4%
Validation accuracy: 9.4%
Minibatch loss at step 100: 1.203898
Minibatch accuracy: 67.2%
Validation accuracy: 66.7%
Minibatch loss at step 200: 1.093140
Minibatch accuracy: 73.4%
Validation accuracy: 80.1%
Minibatch loss at step 300: 1.084395
Minibatch accuracy: 71.9%
Validation accuracy: 81.7%
Minibatch loss at step 400: 0.733119
Minibatch accuracy: 78.1%
Validation accuracy: 83.6%
Minibatch loss at step 500: 0.483472
Minibatch accuracy: 87.5%
Validation accuracy: 84.3%
Minibatch loss at step 600: 0.456564
Minibatch accuracy: 87.5%
Validation accuracy: 85.1%
Minibatch loss at step 700: 0.549403
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 800: 0.423047
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Minibatch loss at step 900: 0.539150
Minibatch accuracy: 82.8%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.548766
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
M